In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from pathlib import Path
import pymc as pm
import arviz as az
import multiprocessing as mp
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng
# from autumn.infrastructure.remote import springboard
from tbdynamics.camau.calibration.utils import get_bcm
from estival.utils.sample import SampleTypes

In [ ]:
covid_configs = {
        'no_covid': {
            "detection_reduction": False,
            "contact_reduction": False
        },  # No reduction
        'detection': {
            "detection_reduction": True,
            "contact_reduction": False
        },  # No contact reduction
        'contact': {
            "detection_reduction": False,
            "contact_reduction": True
        },  # Only contact reduction
        'detection_and_contact': {
            "detection_reduction": True,
            "contact_reduction": True
        },  # With detection + contact reduction
    }

covid_effects = {
    'detection_reduction':True,
    'contact_reduction':False
}
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
N_CHAINS = 8

In [ ]:
def calibrate(out_path, params, covid_effects, draws, tune):
    bcm = get_bcm(params, covid_effects, homo_mixing=False)
    def optimize_ng_with_idx(item):
        idx, sample = item
        opt = eng.optimize_model(bcm, budget=2000, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=8)
        rec= opt.minimize(2000)
        return idx, rec.value[1]

    lhs_samples = bcm.sample.lhs(16, ci=0.67)
    lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
    lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
    opt_samples_idx = map_parallel(optimize_ng_with_idx, lhs_sorted.iterrows())
    best_opt_samps = bcm.sample.convert(opt_samples_idx)
    init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0:N_CHAINS]
    n_chains = N_CHAINS
    with pm.Model() as pm_model:
        variables = epm.use_model(bcm)
        idata_raw = pm.sample(
            step=[pm.DEMetropolisZ(variables, proposal_dist=pm.NormalProposal)],
            draws=draws,
            cores= 16,
            tune=tune,
            discard_tuned_samples=False,
            chains=n_chains,
            progressbar=True,
            initvals=init_samps,
        )
    idata_raw.to_netcdf(str(out_path / "calib_full_out.nc"))

In [ ]:
def calibrate_with_configs(out_path, params, covid_configs, draws, tune):
    for config_name, covid_effects in covid_configs.items():
        # Call the original calibrate function with each covid_effects
        bcm = get_bcm(params, covid_effects)
        
        def optimize_ng_with_idx(item):
            idx, sample = item
            opt = eng.optimize_model(bcm, budget=500, opt_class=ng.optimizers.TwoPointsDE, suggested=sample, num_workers=8)
            rec = opt.minimize(500)
            return idx, rec.value[1]

        lhs_samples = bcm.sample.lhs(16, ci=0.67)
        lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
        lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
        opt_samples_idx = map_parallel(optimize_ng_with_idx, lhs_sorted.iterrows())
        best_opt_samps = bcm.sample.convert(opt_samples_idx)
        init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0:4]
        n_chains = N_CHAINS
        
        with pm.Model() as pm_model:
            variables = epm.use_model(bcm)
            idata_raw = pm.sample(
                step=[pm.DEMetropolisZ(variables)],
                draws=draws,
                cores=16,
                tune=tune,
                discard_tuned_samples=False,
                chains=n_chains,
                progressbar=True,
                initvals=init_samps,
            )
        
        # Save results using the configuration key in the filenames
        idata_raw.to_netcdf(str(out_path / f"calib_full_out_{config_name}.nc"))


In [ ]:
OUT_PATH = Path.cwd().parent.parent / 'data/outputs/camau/r1003'

In [ ]:
OUT_PATH 

In [ ]:
draws= 3000
tune = 1500
calibrate(OUT_PATH,params, covid_effects, draws, tune)